In [1]:
import pandas as pd
import numpy as np
import random
import string
import pickle
import os.path
from utils import *

In [2]:
if not os.path.isfile('data/trigrams.pickle'): # run once
    train = open('data/train.txt', 'r') # conll 2003 dataset
    freqs = open('data/freqs.txt', 'r')

    trigrams = trigram_prob(freqs)
    with open('data/trigrams.pickle', 'wb') as handle:
        pickle.dump(tri_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    freqs.seek(0)

    emissions = emission_parameter(freqs)
    with open('data/emissions.pickle', 'wb') as handle:
        pickle.dump(e_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    freqs.seek(0)

    wordc = word_count(freqs)
    with open('data/wordc.pickle', 'wb') as handle:
        pickle.dump(word_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

    wnet = []
    with open("data/data.noun", "r") as file:
        lines = file.readlines()
        for l in lines:
            if l != "":
                wnet.append(l.split(' ')[4])

    caps = []
    for w in wnet:
        if w != "":
            asc = ord(w[0])
            if asc >= 65 and asc < 91:
                caps.append(w.replace('_', ' '))

    with open('data/wnet_caps.pickle', 'wb') as handle:
        pickle.dump(caps, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
else:
    with open('data/trigrams.pickle', 'rb') as handle:
        trigrams = pickle.load(handle)
    
    with open('data/emissions.pickle', 'rb') as handle:
        emissions = pickle.load(handle)

    with open('data/wordc.pickle', 'rb') as handle:
        wordc = pickle.load(handle)

    with open('data/wnet_caps.pickle', 'rb') as handle:
        caps = pickle.load(handle)

In [3]:
# read dataset
ansar = pd.read_csv("../../Dataset/Ansar1.txt", sep='\t')
ansar = ansar[ansar['Message'].notna()] # drop nan value rows
messages = dict(zip(ansar['MessageID'], ansar['Message']))

In [4]:
def processMessage(message):
    '''
        Takes message, returns english only tokens.
    '''
    printables = filter(lambda x: True if x==' ' else x in string.printable, message)
    eng_text = ""
    for ch in printables:
        eng_text += ch
        
    tokens = nltk.word_tokenize(eng_text)
    for i in range(len(tokens)):
        if tokens[i] is '.':
            tokens.insert(i+1, '')

    if tokens and tokens[-1]:
        tokens.append('')
    
    return tokens

# Similarity Scores

In [5]:
with open('data/sim_scores.pickle', 'rb') as handle:
    sim_scores = pickle.load(handle)

In [10]:
# 
# user1 and user2 are both MemberID
for user1 in sim_scores: # dict
    for user2 in sim_scores[user1]: #dict
        print("{} to {} sim: {}".format(user1, user2, sim_scores[user1][user2])) # ner weight between user1 and user2
    break

12 to 1060 sim: 0
12 to 54 sim: 10.243288799607198
12 to 932 sim: 1.8823818375683685
12 to 148 sim: 0.33012196967279545
12 to 10 sim: 3.392557199138896
12 to 137 sim: 0.1122632861192121
12 to 930 sim: 33.16350885301043
12 to 139 sim: 0
12 to 868 sim: 0.20514404734276634
12 to 151 sim: 0
12 to 7 sim: 0.005658414800576942
12 to 1 sim: 17.931738550757203
12 to 74 sim: 0.0718244030931087
12 to guest1 sim: 0.438620647748129
12 to 343 sim: 0.26352830985103143
12 to 33 sim: 0.0008787675549332735
12 to fighter sim: 3.4805706774580445
12 to 936 sim: 0.08785693512391446
12 to 825 sim: 0
12 to 714 sim: 0.29865023479017355
12 to 353 sim: 0.1778452541504907
12 to 109 sim: 1.3719024455668989
12 to 116 sim: 0.0025335176270659954
12 to 991 sim: 0.0002027086949360834
12 to 534 sim: 0.518996462900969
12 to 171 sim: 0
12 to 53 sim: 0.01228413307726922
12 to 1071 sim: 0.002477212906822661
12 to 13 sim: 0
12 to 154 sim: 0.9915133595727619
12 to 917 sim: 6.334646716752607e-05
12 to 331 sim: 5.06771737340208

In [12]:
# same weight stored bidirectionally
print(sim_scores['54']['12'])
print(sim_scores['12']['54'])

10.243288799607198
10.243288799607198
